## 1. Transit Score

In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import requests 

**Transit Score Documentation:** https://www.walkscore.com/professional/public-transit-api.php <br>
**Google Docs of Support Places:** https://docs.google.com/spreadsheets/d/1pz8GtZKEPISjq-pbYzfRJRD59nLI-_VGgLZiWFAv4gw/edit?hl=en&hl=en#gid=0 <br>
**Requests Documentatiion:** https://pypi.org/project/requests/

In [20]:
def get_data(lat, lng):
    '''
    Gets data for one location.
    
    Inputs:
        lat, lng (str)
    Returns:
        (json)
    '''
    rv = None
    #key = '4c4eb18a1eea25128110eaf683aefab4'
    key = 'ffd1c56f9abcf84872116b4cc2dfcf31' #Mike's first key
    url = 'https://transit.walkscore.com/transit/score/?lat={}&lon={}&wsapikey={}'.format(lat, lng, key)
    r = requests.get(url)
    if not r.status_code==400:
        rv = r.json()
    
    return rv

In [3]:
#Example for Logan Sqaure
get_data('41.9231', '-87.7093')

{'transit_score': 70,
 'help_link': 'https://www.redfin.com/how-walk-score-works',
 'summary': '9 nearby routes: 7 bus, 2 rail, 0 other',
 'logo_url': 'https://cdn.walk.sc/images/transit-score-logo.png',
 'ws_link': 'https://www.walkscore.com/score/loc/lat=41.9231/lng=-87.7093/?utm_source=uchicago.edu&utm_medium=ts_api&utm_campaign=ts_api',
 'description': 'Excellent Transit'}

In [4]:
#Example invalid lat, long
get_data('23', '-56')

In [22]:
def create_datalist(df):
    '''
    Turns list of jsons into a dataframe
    
    Inputs:
        acsse data frame (pandas Dataframe)
    Returns:
        list containing transit score for all places for which it is available (list of json)
    '''
    datalist = []
    
    for row in df.itertuples():
        data = get_data(row.Lat, row.Lon)
        if not data is None:
            data["city"] = row.city
            datalist.append(data)
    
    return datalist

In [6]:
def create_dataframe(data_list):
    '''
    Given the list of json objects containing transit score data, create dataframe.
    '''    
    df = pd.DataFrame.from_dict(json_normalize(data_list), orient='columns')
    df = df.drop(columns=['help_link', 'logo_url'])
    df[['nearby_routes', 'bus', 'rail', 'other']] = df['summary'].str.findall(r'\d+').apply(pd.Series)
    
    return df

## 2. Loading Census Data

Here, loaded on "city" level to align with transit score data and export to csv.

In [7]:
import censusdata

#Loading raw data
acsse = censusdata.download("acsse", 2018, censusdata.censusgeo([("state", "17"), ("place", "*")]), \
                            ["K200101_001E",  "GEO_ID"]).reset_index()

#Manipulations for clarity of columns
acsse = acsse.rename(columns={"K200101_001E": "Population", "index": "censusgeo"})
acsse["censusgeo"] = acsse["censusgeo"].astype(str)

#Isolate place name and place type from censusgeo object
acsse["Location"] = acsse["censusgeo"].str.extract(r'^(.+?),')

#Creating separate columns for place name and place type
acsse["Place_Name"] = acsse["Location"].str.extract(r'(.+)\b\w+$')
acsse["Place_Type"] = acsse["Location"].str.extract(r'\b(\w+)$')

acsse["Place_Name"] = acsse["Place_Name"].str.strip()
acsse["Place_Type"] = acsse["Place_Type"].str.strip()

In [8]:
acsse.shape

(128, 6)

In [9]:
acsse.head()

,censusgeo,Population,GEO_ID,Location,Place_Name,Place_Type
0,"Wheeling village, Illinois: Summary level: 160...",38877,1600000US1781087,Wheeling village,Wheeling,village
1,"Glen Ellyn village, Illinois: Summary level: 1...",27558,1600000US1729756,Glen Ellyn village,Glen Ellyn,village
2,"Normal town, Illinois: Summary level: 160, sta...",55152,1600000US1753234,Normal town,Normal,town
3,"DeKalb city, Illinois: Summary level: 160, sta...",43888,1600000US1719161,DeKalb city,DeKalb,city
4,"Collinsville city, Illinois: Summary level: 16...",25400,1600000US1715599,Collinsville city,Collinsville,city


## 3. Linking Census "Places" with Longitude and Latitude

**Link to US Cities Database**: https://simplemaps.com/data/us-cities <br>
In my directory, it is entitled "uscities.csv". I've added "uscities.csv" to our repo.

In [10]:
#Load supported cities data
cities = pd.read_csv("supported_cities.csv")
#Look at only Illinois cities
il_cities = cities[cities['state']=='IL']

In [11]:
il_cities.shape

(68, 8)

In [36]:
il_cities.head(5)

,city,state,country,population,Lat,Lon,hoods_city_id,hoods_city_name
3,Chicago,IL,NaN,2853114,41.878114,-87.629798,3.0,Chicago
203,Wheaton,IL,NaN,52894,41.865180,-88.106506,720.0,Wheaton
205,Oak Park,IL,NaN,52104,41.884873,-87.787354,738.0,Oak Park
219,Glenview,IL,NaN,44692,42.068410,-87.794666,883.0,Glenview
221,Elmhurst,IL,NaN,43996,41.898925,-87.941141,890.0,Elmhurst


In [13]:
il_cities[il_cities["city"]==acsse["Place_Name"][0]]

,city,state,country,population,Lat,Lon,hoods_city_id,hoods_city_name
847,Wheeling,IL,NaN,37648,42.138668,-87.929226,1051.0,Wheeling


In [14]:
#Merging acsse and il_cities
merged = pd.merge(acsse, il_cities, how="inner", left_on="Place_Name", right_on="city")

In [15]:
#Same shape as acsse, so all entries are matched
merged.shape

(53, 14)

In [17]:
merged.head(2)

,censusgeo,Population,GEO_ID,Location,Place_Name,Place_Type,city,state,country,population,Lat,Lon,hoods_city_id,hoods_city_name
0,"Wheeling village, Illinois: Summary level: 160...",38877,1600000US1781087,Wheeling village,Wheeling,village,Wheeling,IL,NaN,37648,42.138668,-87.929226,1051.0,Wheeling
1,"Glen Ellyn village, Illinois: Summary level: 1...",27558,1600000US1729756,Glen Ellyn village,Glen Ellyn,village,Glen Ellyn,IL,NaN,27450,41.876289,-88.067608,1504.0,Glen Ellyn


In [23]:
#Create list of jsons returned by transit score API
datalist = create_datalist(merged[['city', 'Lat', 'Lon']])

In [24]:
#Turn transit score list into dataframe
ts = create_dataframe(datalist)
#Check that all cities in merged get a transit score 
ts.shape

(53, 9)

In [40]:
#Cleanup on final dataframe to be exported as csv
tsdf = pd.merge(ts, merged, how="inner", left_on="city", right_on="Place_Name")
tsdf = tsdf.rename(columns={'city_x': 'city'})
tsdf = tsdf.drop(columns=['Location', 'Place_Name','city_y','country','Population', 'population', 'hoods_city_id',
       'hoods_city_name'])

In [44]:
#Export as csv
tsdf.to_csv('transit_score.csv', index=False)

In [60]:
#Checking rows that didn't match for transit score
check = acsse.merge(il_cities, how='left', left_on="Place_Name", right_on="city", indicator=True)

acs_only = check[check['_merge']=='left_only']
ts_only = check[check['_merge']=='right_only']

print('Number of transit score supported cities incorrectly left out: {}'.format(ts_only.shape[0]))
acs_only.head()

Number of transit score supported cities incorrectly left out: 0


,censusgeo,Population,GEO_ID,Location,Place_Name,Place_Type,city,state,country,population,Lat,Lon,hoods_city_id,hoods_city_name,_merge
4,"Collinsville city, Illinois: Summary level: 16...",25400,1600000US1715599,Collinsville city,Collinsville,city,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
5,"Bartlett village, Illinois: Summary level: 160...",42135,1600000US1704013,Bartlett village,Bartlett,village,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
7,"West Chicago city, Illinois: Summary level: 16...",29091,1600000US1780060,West Chicago city,West Chicago,city,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
9,"Lockport city, Illinois: Summary level: 160, s...",25785,1600000US1744225,Lockport city,Lockport,city,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
10,"Elk Grove Village village, Illinois: Summary l...",33850,1600000US1723256,Elk Grove Village village,Elk Grove Village,village,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
